In [41]:
# Imports 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, \
    confusion_matrix, classification_report

from sklearn.metrics import roc_auc_score, plot_roc_curve, roc_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve,plot_precision_recall_curve

from sklearn.model_selection import learning_curve

from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

from sklearn.inspection import permutation_importance
#import warnings
#warnings.simplefilter(action="ignore")

# Baseline model

In [42]:
immo = pd.read_csv("immo.csv")

In [43]:
# define X, y
y = immo["ocean_proximity_number"]
X = immo.drop("ocean_proximity_number", axis=1)

In [44]:
# instantiate Dummy classifier
dummy_clf = DummyClassifier(strategy="most_frequent")

# fit the modem
dummy_clf.fit(X, y)

# calculate the score
dummy_clf.score(X, y)

0.43564292201894383

---Now we can see the score of my baseline: 0.43---

In [45]:
# instantiate Dummy classifier
dummy_clf = DummyClassifier(strategy="most_frequent")

# fit the modem
dummy_clf.fit(X, y)

# calculate the score
dummy_clf.score(X, y)

0.43564292201894383

---We can see a little improvement with the dataset : immo_with_outliers_and_imput and immo_with_outliers---

---for now i'll keep the outliers---

# First iteration

In [46]:
immo_with_imput = pd.read_csv("immo_with_imput.csv")

In [47]:
def Lineareg(data):
    y = data["median_house_value"]
    X = data.drop("median_house_value", axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1)
    model = LinearRegression()
    model.fit(X_train, y_train)
    print(f'model score: {model.score(X_test, y_test)}')

    K = []
    total_time = []
    score = []
    for k in range(2,20):
        cross_val_results = cross_validate(model, X, y, cv=k)
        total_time.append(sum(cross_val_results['fit_time'])+sum(cross_val_results['score_time']))
        K.append(k)
        score.append(cross_val_results['test_score'].mean())
    best_cv = pd.DataFrame({'K': K,'score': score})
    cross = best_cv.query(f'score=={max(score)}')['K'].item()
    cv_results = cross_validate(model,X, y,cv=cross, scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_error'))
    r2 = cv_results['test_r2'].mean()
    rmse = cv_results['test_neg_root_mean_squared_error'].mean()
    print(f'r2: {r2}')
    print(f'rmse: {rmse}')

In [48]:
Lineareg(immo_with_imput)

model score: 0.6058961194280346
r2: 0.6016233887270823
rmse: -56747.678579654465


# Second iteration

In [49]:
immo_with_outliers = pd.read_csv("immo_with_outliers.csv")
immo_with_outliers_and_imput = pd.read_csv("immo_with_outliers_and_imput.csv")

In [50]:
Lineareg(immo_with_outliers)

model score: 0.6324224669228723
r2: 0.6353530950356457
rmse: -69539.75736337178


In [51]:
Lineareg(immo_with_outliers_and_imput)

model score: 0.6324224669228723
r2: 0.6353530950356457
rmse: -69539.75736337178


# Fourth iteration

In [52]:
one_hot_iteration = pd.read_csv("one_hot_iteration.csv")

In [54]:
one_hot_iteration

,Unnamed: 0.1,Unnamed: 0,longitude,latitude,housing_median_age,total_rooms,population,households,median_income,median_house_value,ocean_proximity,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,0,2072,-119.84,36.77,6.0,1853.0,1397.0,417.0,1.4817,72000.0,INLAND,0.0,1.0,0.0,0.0,0.0
1,1,10600,-117.80,33.68,8.0,2032.0,862.0,340.0,6.9133,274100.0,<1H OCEAN,1.0,0.0,0.0,0.0,0.0
2,2,2494,-120.19,36.60,25.0,875.0,931.0,214.0,1.5536,58300.0,INLAND,0.0,1.0,0.0,0.0,0.0
3,3,4284,-118.32,34.10,31.0,622.0,597.0,227.0,1.5284,200000.0,<1H OCEAN,1.0,0.0,0.0,0.0,0.0
4,4,16541,-121.23,37.79,21.0,1922.0,1130.0,372.0,4.0815,117900.0,INLAND,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16507,16507,1099,-121.90,39.59,20.0,1465.0,745.0,250.0,3.0625,93800.0,INLAND,0.0,1.0,0.0,0.0,0.0
16508,16508,18898,-122.25,38.11,49.0,2365.0,1131.0,458.0,2.6133,103100.0,NEAR BAY,0.0,0.0,0.0,1.0,0.0
16509,16509,11798,-121.22,38.92,19.0,2531.0,1206.0,429.0,4.4958,192600.0,INLAND,0.0,1.0,0.0,0.0,0.0
16510,16510,6637,-118.14,34.16,39.0,2776.0,2546.0,773.0,2.5750,153500.0,<1H OCEAN,1.0,0.0,0.0,0.0,0.0


In [53]:
Lineareg(one_hot_iteration)

ValueError: could not convert string to float: 'NEAR BAY'